In [1]:
# @author: lajello
# A collection of examples of how to use APIs and API wrappers.

### Imports

In [1]:
# generic imports useful for all APIs
import requests
import json
import os
import time
import pandas as pd
import bz2

### Marvel API

Create and account on https://developer.marvel.com/. In the section "Get a Key", you can generate a new public/privare key pair. 

Go to https://developer.marvel.com/docs to see the full specification of API calls

In [16]:
import hashlib
from datetime import datetime

In [17]:
# the main API endpoint
marvel_endpoint = 'https://gateway.marvel.com/'
# endpoints for specific API calls
marvel_endpoint_comics = marvel_endpoint+'v1/public/comics'
marvel_endpoint_characters = marvel_endpoint+'v1/public/characters'

# ADD VALUES HERE
# the API keys -- you will need them both to compose the API requests
marvel_public_key = 'e97d46a6b2bcb928ce3f75107099e777'
marvel_private_key = 'a2bca05af0355f52532fdcc9700130dfe9395fd7'

In [35]:
def get_hash(marvel_private_key, marvel_public_key):
    '''
    API requests from "server-side" (meaning: requests coming from a script rather than a website or app)
    require an additional hash to be sent in the GET request. This function calculates this hash
    @param marvel_private_key: your API private key
    @param marvel_public_key: your API public key
    @return a pair (timestamp, hex string representing the hash)
    '''
    # get the current time stamp
    ts = int(datetime.now().timestamp())
    # calculates the hash of the concatenation of ts and keys
    hashcode = hashlib.md5(f'{ts}{marvel_private_key}{marvel_public_key}'.encode()).hexdigest()
    print(hashcode)
    return (ts,hashcode)

#### Get comics

In [39]:
ts, hashcode = get_hash(marvel_private_key, marvel_public_key)

# sends an http GET request with a number of additional parameters in the URL
response = requests.get(
    marvel_endpoint_comics,
    params={'apikey': marvel_public_key,
            'hash': hashcode,
            'ts': ts,
            'offset': 0,
            'format': 'comic',
            'formatType': 'comic',
            'dateRange': '2013-01-01,2013-01-02'
            }
    )
#print("Response",response)

59b19a72c26e914ee5c547794fc73313


In [40]:
# the response is returned as a stream of bytes (we will discuss encodings in one of the next lessons)
json_response = json.loads(response.content.decode('utf8'))

# this is how json responses are saved on file
with open('outf.json', 'wt') as outf:
    json.dump(json_response, outf)

In [21]:
# check the http response code and status
print(json_response['code'], json_response['status'])
# get the response data
data = json_response['data']
# responses are paginated, here a short explanation of what that means:
print(f'This response contains {data["count"]} records (from index {data["offset"]} to index {data["limit"]})')
print(f'But the total number of records available for this request is {data["total"]}')
print(f'To get additional records, issue a new API request with offset starting at {data["limit"]}')

200 Ok
This response contains 20 records (from index 0 to index 20)
But the total number of records available for this request is 27
To get additional records, issue a new API request with offset starting at 20


In [22]:
# iterate over the results
character_set = set()
for res in data['results']:
    # extract comic title (similar notation for other fields)
    comic_title = res['title']
    # extract set of characters
    characters = res['characters']['items']
    for c in characters:
        # the character ID is the last segment of the character URI
        character_id = c['resourceURI'].split('/')[-1]
        character_name = c['name']
        character_set.add((character_id, character_name))
print('Set of characters:', character_set)

Set of characters: {('1009454', 'Morbius'), ('1009494', 'Pepper Potts'), ('1011436', 'Red She-Hulk'), ('1009262', 'Daredevil'), ('1009663', 'Venom (Flash Thompson)'), ('1017834', 'War Machine (James Rhodes)'), ('1009726', 'X-Men'), ('1009368', 'Iron Man')}


#### Get characters

In [23]:
import shutil

for character_id, character_name in character_set:
    print(character_name)
    ts, hashcode = get_hash(marvel_private_key, marvel_public_key)
    # sends an http GET request with a number of additional parameters in the URL
    response = requests.get(
        f'{marvel_endpoint_characters}/{character_id}',
        params={'apikey': marvel_public_key,
                'hash': hashcode,
                'ts': ts
                }
        )
    # get the thumbnail URL
    json_response = json.loads(response.content.decode('utf8'))
    thumb = json_response['data']['results'][0]['thumbnail']
    thumb_url = f'{thumb["path"]}.{thumb["extension"]}'
    # send a regular http request to get the image
    image = requests.get(thumb_url, stream=True)
    # save image on file
    with open(f'marvel_pics/{character_name}.{thumb["extension"]}', 'wb') as out_file:
        shutil.copyfileobj(image.raw, out_file)

Morbius
Pepper Potts
Red She-Hulk
Daredevil
Venom (Flash Thompson)
War Machine (James Rhodes)
X-Men
Iron Man


#### Marvel API python wrapper
Many API have "wrappers" available in several languages. Wrappers are just pieces of code that manage the http requests and handle the responses (what we did in the code above), so that you can call simple methods to get results directly.

In [32]:
#pip install marvel

In [34]:
conda list python


# packages in environment at /Users/reka/opt/anaconda3:
#
# Name                    Version                   Build  Channel
ipython                   8.2.0            py39hecd8cb5_0  
ipython_genutils          0.2.0              pyhd3eb1b0_1  
msgpack-python            1.0.2            py39hf7b0b51_1  
opencv-contrib-python     4.5.5.64                 pypi_0    pypi
opencv-python             4.5.5.64                 pypi_0    pypi
python                    3.9.12               hdfd78df_0  
python-dateutil           2.8.2              pyhd3eb1b0_0  
python-fastjsonschema     2.15.1             pyhd3eb1b0_0  
python-libarchive-c       2.9                pyhd3eb1b0_1  
python-lsp-black          1.0.0              pyhd3eb1b0_0  
python-lsp-jsonrpc        1.0.0              pyhd3eb1b0_0  
python-lsp-server         1.2.4              pyhd3eb1b0_0  
python-slugify            5.0.2              pyhd3eb1b0_0  
python-snappy             0.6.0            py39h23ab428_3  
python.app             

In [31]:
from marvel import Marvel
m = Marvel(marvel_public_key, marvel_private_key)
characters = m.characters
character = characters.get(1009189)
character

{'code': 200,
 'status': 'Ok',
 'copyright': '© 2022 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2022 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2022 MARVEL</a>',
 'etag': '2a17d066350c6e0d1cde58d0d848f2214100c6ac',
 'data': {'offset': 0,
  'limit': 20,
  'total': 1,
  'count': 1,
  'results': [{'id': 1009189,
    'name': 'Black Widow',
    'description': '',
    'modified': '2016-01-04T18:09:26-0500',
    'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/f/30/50fecad1f395b',
     'extension': 'jpg'},
    'resourceURI': 'http://gateway.marvel.com/v1/public/characters/1009189',
    'comics': {'available': 586,
     'collectionURI': 'http://gateway.marvel.com/v1/public/characters/1009189/comics',
     'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/43495',
       'name': 'A+X (2012) #2'},
      {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/43509',
       'name': 'A+X (2012) #10'},
     

### Reddit API

#### OAuth authentication
Got to https://www.reddit.com/prefs/apps and create a new application

In [10]:
# ADD VALUES HERE
# get these values by registering an application at https://www.reddit.com/prefs/apps
personal_use_script = 'MnFYanKjmsnmqMpyC9qYew'
secret = 'ih3_YmSYrYI4rB4HLoEYlvXDcSBzBQ'

# create an object to be used for http basic authentication
auth = requests.auth.HTTPBasicAuth(personal_use_script, secret)

# ADD VALUES HERE
# username and password to be placed in the token request
data = {'grant_type': 'password',
        'username': 'rekas12',
        'password': '468855@Bruskk....'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'data_in_the_wild'}

# send a POST request for an OAuth token
response = requests.post('https://www.reddit.com/api/v1/access_token',
                         auth=auth, data=data, headers=headers)

print(response.json())
# get the access token from the response
oauth_token = response.json()['access_token']

# add the token to the http request header
# the ** operator map the keys of the two dictionaries and add the values 
# of the second dict to the first dict 
headers = {**headers, **{'Authorization': f'bearer {oauth_token}'}}

# send a request for a resource using the authentication header
response = requests.get('https://oauth.reddit.com/r/wallstreetbets/new', headers=headers)
json_response = response.json()
json_response

{'access_token': '2227631114692-UNwCz8ECY37G5lPnLI2Inp1IPbGJFQ', 'token_type': 'bearer', 'expires_in': 86400, 'scope': '*'}


{'kind': 'Listing',
 'data': {'after': 't3_xo9aj7',
  'dist': 25,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'wallstreetbets',
     'selftext': 'https://www.asiamarkets.com/global-depression-warning-from-analyst/\n\n“Unlike the Great Depression of the 1930s, this (will be) a full-scale globalised depression,” Hunt told Asia Markets.\n\n“It’s a pretty dystopian message .. (but) we’re seeing it across all charts.\n\n“It’s going to be brutal on any form of leverage and it’s going to be an impoverishing event generally for the middle classes.”\n\n“We’ll need to go back and ask ourselves – what are our physical needs?”\n\nThose needs, he says, are the basics – like energy, shelter, water and food.\n\n“Farming has almost been almost demonetized compared to being an investment banker, to such an extreme that we just think of them as dirty people who rummage around in the dirt,” he said.\n\n“Now, boy, are they go

#### Pushshift

Pushift is a service that collects social media data using APIs. It stores data in third-party servers. You can query the pushift API to get Reddit data (no auth required). PMAW is a python wrapper for the Pushift API. 

Pushift API: https://github.com/pushshift/api

PMAW: https://github.com/mattpodolak/pmaw

In [13]:
#pip install pmaw

In [16]:
from pmaw import PushshiftAPI

# create an API wrapper object
# PMAW automatically manages parallelization of requests using multiple 
# proesses running on multiple CPU cores
api = PushshiftAPI(num_workers=os.cpu_count()*5)

# setting query parameters
subreddit = 'wallstreetbets'
before = int(time.time())
after = 0
limit = 100

# get submissions
submissions = api.search_submissions(subreddit=subreddit, limit=limit, before=before, after=after)
sub_df = pd.DataFrame(submissions)

# get comments
comments = api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
comments_df = pd.DataFrame(comments)

Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.


In [ ]:
sub_df.head()

In [ ]:
comments_df.head()

### Twitter API

Go to https://developer.twitter.com/en/apply-for-access and apply for access

Then go to https://developer.twitter.com/en/portal/projects-and-apps, create a project and get the API credentials

API v1.1 docs: https://developer.twitter.com/en/docs/twitter-api/v1

API v2 docs: https://developer.twitter.com/en/docs/twitter-api/early-access

Twitter API rate limits (v2): https://developer.twitter.com/en/docs/twitter-api/rate-limits

Twitter API rate limits (v1.1): https://developer.twitter.com/en/docs/twitter-api/v1/rate-limits

Step-by-step guide for beginners: https://developer.twitter.com/en/docs/tutorials/step-by-step-guide-to-making-your-first-request-to-the-twitter-api-v2

In [2]:
# ADD VALUES HERE
api_key = 'hjQcwo5OaQx5XWU93Fz8jvOmz'
secret = 'xitZgg8gIOsclO0a18VcoHzQr18NM8PHg9KTZTGCHUNNKHeosx'
#bearer_token = ''
access_token = '2312044846-glKiGq7bAhimFQek1N672fH0d7KPh4NK20tF9DR'
access_token_secret = '7C6RCQCpojodcCZUWFzvxVMfefxpMfNezkfpuZkjMixmO'

#### v1.1 with Tweepy

In [3]:
import tweepy

# OAuth authentication
auth = tweepy.OAuth1UserHandler(api_key, secret, access_token, access_token_secret)
api = tweepy.API(auth)

In [4]:
# get tweets from own timeline
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

Hello from Political Science Research5
Hello from Political Science Research4
RT @timnitGebru: @mmitchell_ai @SamaAI I'm not sure as this is what Sama did most recently. 
https://t.co/UWo2VBkBYI
RT @KenBurns: If you missed The US and the Holocaust it’s streaming for free on https://t.co/iapz56gCbZ.  Hope you have a chance to watch.…
@SamaAI, bet you can make this happen!
A basic point in this article: If you're using a country like Kenya for data labeling and justify to yourself that… https://t.co/E2yH9qSQKY
"performance and licensing information were the two most important areas respondents wanted to see in model cards."
I love this!
"Based on our research, existing model cards should be enhanced in two primary areas: accessibility an… https://t.co/yfZQ7NAZhm
RT @AnimaAnandkumar: Check out new model cards++ by @NVIDIAAI https://t.co/6JX8PGbQU8 building on model cards by @mmitchell_ai @timnitGebru…
RT @timnitGebru: A @ruthstarkman  appreciation tweet. A woman with endless energy and ho

In [5]:
# streaming with API v1.1 (soon to be deprecated)
class MyStream(tweepy.Stream):
    """
    Extend the tweepy.Stream API to do something with the tweets received
    """
    def on_status(self, status):
        print(status.text)

# get all tweets containing a given keyword
stream = MyStream(api_key, secret, access_token, access_token_secret)
stream.filter(track=['ukraine'])

RT @NEWS_ALL_TIME: Russian cop flees from the protesters in Makhachkala, Dagestan, North Russia.
#Russia #RussiaInvadedUkraine #Ukraine #Uk…
RT @FiorellaIsabelM: 50,000 Muscovites rallied to support the referendum. Westerners don’t realize the whole reason this conflict’s happeni…
RT @holosUA: Дагестан, окупована рф кавказька країна

Жінки прямо в очі поліції нагадують, що росія напала на Україну, а не навпаки.

в'язн…
RT @Gerashchenko_en: Mobilized man killed a drafting office commander in Ust-Ilimsk, Irkutsk region, Russia. 

Alexandr Yeliseev, the comma…
RT @mishacollins: “A.I… am your father.” 

https://t.co/28CBNhFINN
RT @BruceFeinEsq: Nigerian elections are as illegitimate as Putin’s sham variety in Ukraine. Nigeria’s boundaries were illegally demarcated…
@blue_jaunt @UAWeapons I mean they were right. Looks like Russia just decided to drive it into Ukraine and leave it… https://t.co/qhYaFkREOZ
I coscritti appena mobilitati 
in #Russia, senza alcun addestramento militare, 
vengono

RT @KathyMcKayNB: UNREAL! Trudeau states the Govt will "match" donations Canadians make to the Red Cross for Hurricane relief in the Mariti…
RT @MoniquePlaza3: Une du Guardian il y a huit ans. «En Ukraine, les 🇺🇸 nous entraînent dans une guerre avec la Russie. Le rôle joué par Wa…
RT @ARybinska: Dyrektor zarządzający Energie Waldeck-Frankenberg (EWF), dostawcy energii w niemieckiej Hesji, pojechał obserwować referendu…
https://t.co/R6elsEImCh
RT @MacaesBruno: So many protests have failed, so many struggles for freedom have come to nothing. So many just wars have been lost. So man…
RT @FCOUILBAULT1: Après avoir "travaillé" pour les labos on peut se demander si #VonDerLeyen ne travaille pas pour l'industrie de l'armemen…
RT @9Parabellum9mm: Avez vous remarqué que l’#Ukraine en plus d’être le 1er producteur mondial de tout est aussi le seul pays au monde oû i…
RT @KyivIndependent: ⚡️ UK intelligence: Russia's mobilized troops to suffer 'high attrition rate' in Ukraine.

The first men enli

RT @KyivPost: In #Zelensky’s latest address on Sunday evening, September 25, the Ukrainian President stated that Criminal mobilization is u…
After months of war, many Ukrainians fear more suffering and political repression awaits them. Kremlin-orchestrated… https://t.co/8jHEQ2lj3z
RT @mhmck: Heads of government who cannot say they want Ukraine to win the war and the russian fascist invaders to be defeated are not fit…
RT @HiroGallery: The Czech business mission #businessklubukrajina is on its way to Lviv, Ukraine. Continuity of international trade is a ke…
RT @johnpilger: I wrote this for the Guardian in 2014 when the coup junta in Kiev declared war on its Russian-speaking population in the Do…
RT @shaunwalker7: 2014 marches for peace in central Moscow, tens of thousands of people came. In the first pic is one of the organisers, Bo…
RT @zozo_64947282: Ukraine has been seeking membership in the EU and NATO for over a decade. But it was not allowed. In addition to the wea…
RT @nata_druha

Ukraine’s Zelenskyy says two more burial sites found in liberated Izium https://t.co/9MSSd0Mu7s
RT @lemondefr: 𝗟𝗜𝗩𝗘 #Ukraine | « Nous avons été très clairs avec les Russes, publiquement et en privé, pour qu’ils cessent de parler d’arme…
The globalists NEED the Ukranian conflict to further the agenda of forming a New World Order, global government.  T… https://t.co/MR5NnsPHrs
En UKRAINE, des anarchistes montent au front pour combattre l’invasion russe

https://t.co/0OfomJjXvI… https://t.co/Gs5jtVDuIP
RT @Ludachrislv: Signer la pétition pour que la russie ne siège plus au Conseil de Sécurité de l’ONU, la place qu’elle a piquée à l’URSS sa…
Monstrueuse !
RT @Boba12340769066: "Hi. If possible, just retweet the photo of my father who was killed in action. I want him at least to be seen and und…
RT @SethAbramson: Putin has hit maximum crazy: he’s going to close Russia’s borders so the nation becomes a prison for conscription-age men…
RT @volkskamera: Die Hoffnung stirbt zuletzt.... 

Putin h

RT @Cutwitts: @Flash_news_ua Never mind. #Ukraine will win this war, no matter what!
RT @KyivIndependent: ⚡️ Blinken: US has plan in place if Russia uses nuclear weapons against Ukraine. 

U.S. Secretary of State Antony Blin…
Guerre en Ukraine | Près de 300 écoles détruites par l'armée russe depuis février https://t.co/M2a9oPUdLa via @lp_lapresse
RT @Ukraine: «We are not blind! Russia attacked Ukraine. Russia is on foreign soil. No to war!” Dagestan women protest Putin’s criminal mob…
RT @andersostlund: I really recommend you follow Michael MacKay @mhmck. He has reported on the war in Ukraine since 2014. His dedication is…
The thing I like most about Sergei Lavrov is his irrepressible sense of humor. He's the life of the party wherever… https://t.co/cB2jtsz5Pa
RT @avalaina: Impressive photo from protest in Russia against mobilisation. The victory of Ukraine will give a chance for democratic future…
RT @Boba12340769066: "Hi. If possible, just retweet the photo of my father who was kille

...

RT @McFaul: While Ukraine has the momentum, this is the moment for the United States, NATO, and other partners to go all in — more and bett…
RT @Gerashchenko_en: Mobilized man killed a drafting office commander in Ust-Ilimsk, Irkutsk region, Russia. 

Alexandr Yeliseev, the comma…
2/2
L'assassino è un Ruslan Zinin "parzialmente grave" classe 1998, che ha deciso che sarebbe stato meglio andare i… https://t.co/Egdft2JlNo
RT @GraniTweet: Эксперты ООН подтвердили факты изнасилования рашистами маленьких детей https://t.co/WyToyfU3UU
RT @Flash_news_ua: ⚡️Kazakhstan does not recognize the results of the "referendums" in the occupied territories of the south and east of Uk…
My Scholz shows the world what leadership is by providing a striking counter-example.
RT @ManUnitedZone_: Scott McTominay will be suspended for the final Nations League game against Ukraine after receiving another yellow card…
RT @mahtab_nmvr: اکراینی‌های عزیز با همه مشکلاتی که دارن هم باز حواسشون بهمون هست 🥹 این همدلی قشنگ

RT @cryptodrftng: Drftng with Ukraine: "Day 214. A Hero"

In memory of Roman Lesnyi

1979 - 2022 🕯❤️

Герої не вмирають.

https://t.co/DqEk…
RT @DefenceHQ: "On day 211 of a five-day operation, none of Russia’s initial objectives have been achieved.”
 
Armed Forces Minister @JSHea…
RT @Flash_news_ua: ⚡️Kazakhstan does not recognize the results of the "referendums" in the occupied territories of the south and east of Uk…
RT @mhmck: Heads of government who cannot say they want Ukraine to win the war and the russian fascist invaders to be defeated are not fit…
RT @nexta_tv: #Russian losses according to the General Staff of the Armed Forces of #Ukraine. https://t.co/rSWZ6KY6hw
RT @JohnEzeakolam: #Russian man bad #Ukraine man good says the man who is liable for the genocide perpetrated in Biafra land in the one-sid…
RT @base10_: Hao bandits ni bad news bana, watafutwe wapeleke NYS wareform then wakuwe absorbed into KDF ata Ukraine na Putin watatuogopa.
In food and nutrition policy this week,

RT @AlexKokcharov: #Kazakhstan's MFA said publicly that the country does not recognise "referendums" in four regions of #Ukraine partially…
RT @KyivIndependent: ⚡️ Blinken: US has plan in place if Russia uses nuclear weapons against Ukraine. 

U.S. Secretary of State Antony Blin…
Drone attack hits Ukraine; US vows 'consequences' over nukes https://t.co/WC2MG6TAU6
RT @KremlinTrolls: Hearing all retired NATO, US, UK commanders say the U.S. needs to lift the weapons embargo on Ukraine for long-range pre…
RT @jpg2t785: ついにウクライナ側で使われる鹵獲T-62Mが出てきた。
https://t.co/MFViQJEhH8
RT @PeterColumns: Programme de la coalition de droite en Italie. Natalisme qui cible les couples actifs. Strict sur l'immigration illégale,…
RT @Liz_Cheney: Must listen ⁦@thedispatch⁩ podcast with ⁦Frmr Ukrainian Finance Minister ⁦@n_jaresko⁩. Natalie &amp; I met 30 yrs ago when we w…
RT @markcurtis30: You might think it would difficult for UK journalists covering Russian war crimes in Ukraine not to mention UK complicity…


@franceinfo https://t.co/soKh8Ck9JS
RT @deadcatssociety: 러시아가 점령한 마을을 구해낸 우크라이나 병사가 엄마와 재회하다

https://t.co/BcyfTLcgvo

"내 사랑! 나의 사랑! 내가 널 위해 기다리고 있었다! 알고 있었어; 네가 올 줄 알았어, 내 꽃이…
RT @tprincedelamour: Rappel: Zelensky à mobilisé 15 Millions d'hommes ukrainiens âgés entre 18 et 60 ans, sous peine de 15 ans de prison.
P…
RT @Ukraine: «We are not blind! Russia attacked Ukraine. Russia is on foreign soil. No to war!” Dagestan women protest Putin’s criminal mob…
@sallywilts Do you think we are in financial crisis in the west now due to Ukraine or lockdowns &amp; “stealth” lockdow… https://t.co/U6AlpQgMl4
RT @jpg2t785: ついにウクライナ側で使われる鹵獲T-62Mが出てきた。
Jake Sullivan: US will act ‘decisively’ if Russia uses nuclear weapons in Ukraine
#US #Putin https://t.co/HQvF6SkHbV
RT @mhmck: Olaf Scholz refusing to send Leopard tanks and Marder infantry fighting vehicles to Ukraine prolongs the war, extends the genoci…
RT @Terror_Alarm: 🚨🇷🇺BREAKING: School Shooting in Izhevsk, #Russia.
At least 1 student was kille

KeyboardInterrupt: 

#### v2 with Requests
Version 2 of the Twitter API has an updated set of endpoints and more functionalities. If you can get an API key in the Academic Research track, it allows you to search for tweets posted at any point in the past (no other method allows you to do that). It is more volume-restricted than v1.1 though. The standard track is limited to 0.5M tweets per month, the Elevated track to 2M per month, and the Academic Research track is limited to 10M per month. 

In [6]:
# API v2 endpoint for recent tweets search 
endpoint_url = 'https://api.twitter.com/2/tweets/search/recent'

# authentication
headers = {'Authorization': f'Bearer {bearer_token}'}

# API v2 allows you to specify the fields to be returned
tweet_fields = ['author_id', 'created_at', 'id', 'text', 'withheld']
tweet_fields = ','.join(tweet_fields)
user_fields = ['created_at', 'description', 'id', 'location', 'name', 'username']
user_fields = ','.join(user_fields)

query = 'ukraine'
query_params = {'query': query,
                'tweet.fields': tweet_fields,
                'user.fields' : user_fields,
                'max_results': 10,
               }
response = requests.request('GET', endpoint_url, headers=headers, params=query_params)
response.json()['data']

NameError: name 'bearer_token' is not defined

#### v2 with Tweepy

In [ ]:
class MyStreamV2(tweepy.StreamingClient):
    def on_data(self, data):
        print(data)

stream = MyStreamV2(bearer_token)
stream.sample()

### Facebook marketing API

Go to https://developers.facebook.com/apps, create an app, create a sandbox account, and get an access token

https://developers.facebook.com/docs/marketing-api/reference/v15.0

https://developers.facebook.com/docs/marketing-apis/get-started

In [ ]:
# ADD VALUE HERE
access_token_sandbox = ''

In [ ]:
query = 'opeth'
query_params = {'q': query,
                'type' : 'adinterest',
                'access_token': access_token_sandbox,
               }
interests = requests.request('GET', 'https://graph.facebook.com/search', params=query_params).json()
interests

### Google speech to text API

Go to https://cloud.google.com/apis and create an account (you will need a credit card for verification). 

You can enjoy 300 USD of free API calls for the first 90 days (checked August 8th 2021). Your card should not be charged after the 90 days if you do not exceed the 300$ and you do not switch to a pro account. After registration, select a new project, add the desired API to the project, and generate a private key, which will be downloaded as a JSON file. Set the name of the json file containing the key as the value of the google_api_credentials_json variable in the cell below.

**IMPORTANT DISCLAIMER: if you would like to the Google Cloud API, you will do it at your own risk. You will be the only person responsible for any incurred costs. Using this API is NOT necessary to pass the exam with full grades.**

Here is the documentation for the python wrapper of the speech-to-text API.
https://cloud.google.com/speech-to-text/docs/libraries#client-libraries-install-python

In [ ]:
import io
import os

# ADD VALUE HERE
# set the json containing the credentials as environment variable
google_api_credentials_json = 'INSERT_FILE_WITH_CREDENTIALS_HERE.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_api_credentials_json
audio_file = 'kennedy_speech.mp3'

#### Main API

In [ ]:
from google.cloud import speech

# instantiate a client
client = speech.SpeechClient()
# read the bytes of the audio file
with io.open(audio_file, 'rb') as f:
    audio_content = f.read()
# create an audio object containing the audio content
audio = speech.RecognitionAudio(content=audio_content)

# set the parameters for the request
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
    sample_rate_hertz=44100,
    language_code='en-US',
    enable_automatic_punctuation=True,
    audio_channel_count=2,
    model='phone_call'
)

# send an API request 
response = client.recognize(config=config, audio=audio)

In [ ]:
response

#### Beta API (some additional functionalities)

In [ ]:
from google.cloud import speech_v1p1beta1 

# instantiate a client
client = speech_v1p1beta1.SpeechClient()
# read the bytes of the audio file
with io.open(audio_file, 'rb') as f:
    audio_content = f.read()
# create an audio object containing the audio content
audio = speech_v1p1beta1.RecognitionAudio(content=audio_content)
# set the parameters for the request
config = speech_v1p1beta1.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
    sample_rate_hertz=44100,
    language_code='en-US',
    enable_automatic_punctuation=True,
    enable_word_confidence=True,
    audio_channel_count=2,
    model='phone_call'
)

# send an API request 
response = client.recognize(config=config, audio=audio)

In [ ]:
response